##02 콘텐츠 기반 필터링 추천 시스템

사용자가 특정한 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식  
Ex) 사용자가 높은 평점을 준 그 영화의 장르, 출연 배우, 감독, 영화 키워드 등의 콘텐츠와 유사한 다른 영화를 추천해주는 방식

##03 최근접 이웃 협업 필터링 / 04 잠재 요인 협업 필터링

협업 필터링 : 사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식(User Behavior)만을 기반으로 추천을 수행하는 것  
주요 목표 : 사용자-아이템 평점 매트릭스와 같은 축적된 사용자 행동 데이터를 기반으로 사용자가 아직 평가하지 않은 아이템을 예측 평가하는 것  



**<사용자-아이템 평점 행렬 특징>** 

*   행 - 개별 사용자
*   열 - 개별 아이템  
*   행,열 위치에 해당하는 값 - 평점
*   많은 아이템을 열로 가지는 다차원 행렬
*   희소 행렬(spare matrix) (사용자가 아이템에 대한 평점을 매기는 경우가 많지 않기 때문)





**<협업 필터링 기반의 추천 시트템의 두 가지 방식>**  
두 방식의 공통점 : 사용자-아이템 평점 행렬 데이터에만 의지해 추천 수행

1.   최근접 이웃 방식(메모리 형업 필터링)


    1.   사용자 기반(User-User) : 당신과 비슷한 고객들이 다음 상품도 구매했습니다.  
         *   특정 사용자와 타 사용자 간의 유사도(Similarity)를 측정한 뒤 가장 유사도가 높은 TOP-NJ 사용자를 추출해 그들이 선호하는 아이템을 추천

    2.   항목 추가(Item-Item) : 이 상품을 선택한 다른 고객들은 다음 상품도 구매했습니다.


        *   '아이템 간의 속성' 이 얼마나 비슷한지를 기반으로 추천하는 것 아님.
        *   사용자들이 그 아이템을 좋아하는지/싫어하는지의 평가 척도가 유사한 아이템을 추천하는 기준이 되는 알고리즘  
        *   Ex) '다크나이트' 는 '스타워즈' 보다 '프로메테우스'와 사용자들의 평점 분포가 훨씬 더 비슷하므로 '다크 나이트'와 '프로메테우스'는 상호 간 아이템 유사도가 상대적으로 매우 높다.  
        따라서 '다크 나이트'를 매우 좋아하는 사용자 D에게 아이템 기반 협업 필터링은 D가 아직 관람하지 못한 '프로메테우스'와 '스타워즈' 중 '프로메테우스'를 추천한다.


    일반적으로 사용자 기반보다는 아이템 기반 협업 필터링이 정확도가 더 높다.   
    이유는 비슷한 영화(또는 상품)를 좋아(또는 구입)한다고 해서 사람들의 취향이 비슷하다고 판단하기는 어려운 경우가 많기 때문이다.
    매우 유명한 영화는 취향과 관계없이 대부분의 사람이 관람하는 경우가 많고, 사용자들이 평점을 매긴 영화(또는 상품)의 개수가 많지 않은 경우가 일반적인데 이를 기반으로 다른 사람과의 유사도를 비교하기가 어려운 부분도 있다.
    따라서 최근접 이웃 협업 필터링은 대부분 아이템 기반의 알고리즘을 적용한다.


2. 잠재 요인 협업 필터링  
사용자-아이템 평점 행렬 속에 숨어 있는 잠재 요인을 추출해 추천 예측을 할 수 있게 하는 기법  
대규모 다찬원 행렬을 SVD 와 같은 차원 감소 기법으로 분해하는 과정에서 잠재 요인을 추출하는데, 이러한 기법을 **행렬 분해(Matrix Factorization)** 

    Ex) 사용자 아이템 평점 행렬 R  = 사용자별 장르 선호도 행렬(P) * 영화별 장르 요소 행렬(Q)의 전치행렬(Q.T)  
    여기서 잠재요인 factor 1 : 액션 / factor 2 : 로맨스  
    사용자가 액션 영화를 매우 좋아하고 특정 영화가 액션 영화의 특성이 매우 크다면 사용자가 해당 영화에 높은 평점을 줄 것이다.

##04-3 확률적 경사 하강법을 이용한 행렬 분해

P와 Q행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추해내는 것  
일반적으로 사용자-아이템 평점 행렬의 경우 행렬 분해를 위해서 단순히 예측 오류값의 최소화와 학습 시 과적합을 피하기 위해서 규제를 반영한 비용 함수를 적용한다.

In [1]:
#행렬 분해
import numpy as np

#원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 k는 3으로 설정
R = np.array([[4,np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])
num_users, num_items = R.shape

In [2]:
R

array([[ 4., nan, nan,  2., nan],
       [nan,  5., nan,  3.,  1.],
       [nan, nan,  3.,  4.,  4.],
       [ 5.,  2.,  1.,  2., nan]])

In [3]:
num_users, num_items

(4, 5)

In [6]:
#P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력한다.
k=3
np.random.seed(1)
P = np.random.normal(scale=1./k,size = (num_users,k))
Q = np.random.normal(scale=1./k,size = (num_items,k))

In [7]:
P

array([[ 0.54144845, -0.2039188 , -0.17605725],
       [-0.35765621,  0.28846921, -0.76717957],
       [ 0.58160392, -0.25373563,  0.10634637],
       [-0.08312346,  0.48736931, -0.68671357]])

In [8]:
Q

array([[-0.1074724 , -0.12801812,  0.37792315],
       [-0.36663042, -0.05747607, -0.29261947],
       [ 0.01407125,  0.19427174, -0.36687306],
       [ 0.38157457,  0.30053024,  0.16749811],
       [ 0.30028532, -0.22790929, -0.04096341]])

In [10]:
#get_rmse() 함수 : 실제 R 행렬과 예측 행렬의 오차를 구함
#실제 R행렬의 널이 아닌 행렬 값의 위치 인텍스를 추출해 이 인텍스에 있는 실제 R행렬 값과 분해된 P,Q를 이용해 다시 조합된 예측 행렬 값의 RMSE 값을 반환
from sklearn.metrics import mean_squared_error

def get_rmse(R,P,Q,non_zeros):
    error = 0
    #두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P,Q.T)

    #실제 R 행렬에서 널이 아닌 값의 위치 인텍스 추출해 실제 R행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind,y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros,full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

SGD 기반으로 행렬 분해 수행한다. 먼저 R에서 널 값을 제외한 데이터의 행렬 인덱스를 추출한다. steps 는 SGD를 반복해서 업데이트할 횟수를 의미하며, learning_rate는 SGD의 학습률, r_lambda는 L2 Regularization 계수

In [13]:
#R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros =[(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

#SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
    for i,j,r in non_zeros:
        #실제 값과 예측 값의 차이인 오류 값 구함.
        eij = r - np.dot(P[i,:], Q[j,:].T)
        #Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R,P,Q,non_zeros)
    if (step % 50) == 0:
        print('### iteration step : ',step,'rmse :', rmse)

### iteration step :  0 rmse : 3.2388050277987723
### iteration step :  50 rmse : 0.4876723101369648
### iteration step :  100 rmse : 0.1564340384819247
### iteration step :  150 rmse : 0.07455141311978046
### iteration step :  200 rmse : 0.04325226798579314
### iteration step :  250 rmse : 0.029248328780878973
### iteration step :  300 rmse : 0.022621116143829466
### iteration step :  350 rmse : 0.019493636196525135
### iteration step :  400 rmse : 0.018022719092132704
### iteration step :  450 rmse : 0.01731968595344266
### iteration step :  500 rmse : 0.016973657887570753
### iteration step :  550 rmse : 0.016796804595895633
### iteration step :  600 rmse : 0.01670132290188466
### iteration step :  650 rmse : 0.01664473691247669
### iteration step :  700 rmse : 0.016605910068210026
### iteration step :  750 rmse : 0.016574200475705
### iteration step :  800 rmse : 0.01654431582921597
### iteration step :  850 rmse : 0.01651375177473524
### iteration step :  900 rmse : 0.016481465738

In [15]:
pred_matrix = np.dot(P,Q.T)
print('예측 행렬:\n', np.round(pred_matrix,3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


원본 행렬과 비교해 널이 아닌 값은 큰 차이가 나지 않으며, 널인 값은 새로운 예측값으로 채워졌다.